# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902060


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:01<00:24,  1.20it/s]

Rendering models:  13%|█▎        | 4/31 [00:02<00:20,  1.34it/s]

Rendering models:  19%|█▉        | 6/31 [00:06<00:28,  1.14s/it]

Rendering models:  32%|███▏      | 10/31 [00:07<00:17,  1.23it/s]

...

Rendering models:  77%|███████▋  | 24/31 [00:08<00:00,  8.32it/s]

Rendering models:  84%|████████▍ | 26/31 [00:08<00:00,  9.27it/s]

Rendering models:  90%|█████████ | 28/31 [00:08<00:00, 10.53it/s]

Rendering models:  97%|█████████▋| 30/31 [00:08<00:00, 11.52it/s]

equidad1                              0.000288
Linda_J._Berkel                       0.000227
briallen                              0.002366
not-logged-in-8c0c44faf754d344b18a    0.007066
Thedunkmasta                          0.001159
not-logged-in-b7e9e8b8644c37156696    0.001570
ElisabethB                            0.000403
JEBala                                0.000227
not-logged-in-ebb378197a91d7a3bbe2    0.000541
not-logged-in-a0164d241f30411408f0    0.000254
AlphaCentaurii                        0.002560
not-logged-in-fc4681569605a6f4f1b0    0.006948
acapirala                             0.006312
rhondacard                            0.002918
Lavadude                              0.007775
clars915                              0.001205
not-logged-in-8700ff8251e0704f745c    0.000184
not-logged-in-4ea23f5fffc8b98635af    0.001084
not-logged-in-0582d7599288fe425356    0.000164
not-logged-in-88d5e424af552d0f2955    0.000991
Lewyke                                0.001688
tdanyluk     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())